[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeljov/NAP2025/blob/main/SNA_Tutorial_Part3.ipynb)

## Detecting communities (clusters) in networks

In [ ]:
!pip install -q igraph

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx
import igraph as ig

import warnings

#### Load the network data

The data comes from the McFarland Classroom Study [1]. It was collected through a sociometric friendship survey and observation of social and task-related interactions of a high-school students in an Algebra II course.

The data includes friendship, social, and task ties between 16 anonymous students:
* friendship: self-reported friendship ties
* social: observed social interactions
* task: observed task interactions

Each type of tie is weighted:
* the friendship ties are weighted: 2 = best friends, 1 = friend, and 0 is not a friend
* social weights reflect the number of observed social interactions per hour
* task weights reflect the number of observed task interactions per hour

[1] McFarland, Daniel A. (2001) “Student Resistance.” American Journal of Sociology 107(3): 612-78. [doi:10.1086/338779](https://ed.stanford.edu/sites/default/files/mcfarland/AJS2001.pdf).

In [ ]:
from google.colab import files

data_file = files.upload()

In [ ]:
net_data = pd.read_csv('McFarland2001_students_multinet.csv')

In [ ]:
# Offline data loading

# from pathlib import Path
# net_data = pd.read_csv(Path.cwd() / 'data' / 'McFarland2001_students_multinet.csv')

In [ ]:
net_data.head()

In [ ]:
net_data.type.value_counts()

Create separate data sources for each network

In [ ]:
social_net = net_data[net_data.type == 'social'].drop(columns=['type']).copy()
task_net = net_data[net_data.type == 'tasks'].drop(columns=['type']).copy()
friendship_net = net_data[net_data.type == 'friends'].drop(columns=['type']).copy()

#### Friendship network

Since zero weight in the friendship network data indicates the absence of connection, we first remove those non-existing ties:

In [ ]:
zero_weight_ties = friendship_net.loc[friendship_net.weight == 0,].index
friendship_net.drop(zero_weight_ties, inplace=True)
friendship_net.reset_index(drop=True, inplace=True)
friendship_net.info()

The way the friendship data was collected (each student naming whom they perceived as friends) suggests that the friendship network should be created as a directed network

In [ ]:
G_friend = nx.from_pandas_edgelist(friendship_net,
                                   source='from',
                                   target='to',
                                   edge_attr='weight',
                                   create_using=nx.DiGraph)
print(G_friend)

Note that 2 nodes are missing (there were 16 actors in total). One of them is the teacher (#16) and the other one (#4) seems to be a student who didn't report any friendship tie (or didn't fill out the survey), nor was recognised as a friend by other students.

To get an idea of what the three student networks look like, we'll create a function that will allow us to quickly plot those networks as graphs.

In [ ]:
def plot_graph(G,
               graph_name,
               node_color_modifiers=None, # will be used for coloring nodes based on the community membership
               edge_weight_multiplier=1 # will be used for plotting edge thickness proportional to the tie strength
               ):
    plt.figure(figsize=(8,8))

    pos = nx.spring_layout(G, seed=9)
    # pos = nx.kamada_kawai_layout(G)

    if node_color_modifiers:
        node_color = [node_color_modifiers[node] for node in G.nodes()]
    else:
        node_color = 'purple'

    edge_width = [attr['weight']*edge_weight_multiplier for (u, v, attr) in G.edges(data=True)]

    nx.draw_networkx_nodes(G, pos, node_size=450, node_color=node_color, cmap='viridis_r')
    nx.draw_networkx_edges(G, pos, width=edge_width, edge_color='steelblue')
    nx.draw_networkx_labels(G, pos, font_color='ivory')

    plt.title(graph_name)

    plt.axis('off')
    plt.show()

In [ ]:
warnings.simplefilter('ignore', UserWarning)

plot_graph(G_friend, "Friendship network", edge_weight_multiplier=1.5)

Since we are interested in the tendency of the network actors to group / cluster, we will start by computing some key measures of actor connectedness and tendency to form groups in a network:
* network density
* local clustering coefficient
* global clustering coefficient (transitivity)

In [ ]:
fr_density = nx.density(G_friend)

print(f"Network density: {fr_density:.3f}")

In [ ]:
fr_trans_score = nx.transitivity(G_friend)

print(f"Friend network's transitivity score: {fr_trans_score:.4f}")

In [ ]:
fr_clust_coef = nx.clustering(G_friend)

pd.Series(fr_clust_coef).describe()

Similar values of the average local clustering and the global clustering coefficients suggest that low connected and high connected nodes (students) do not differ in terms of clustering.

In [ ]:
plot_graph(G_friend,
           "Friendship network, node color denotes local clustering coefficient",
           node_color_modifiers=fr_clust_coef,
           edge_weight_multiplier=1.5)

#### Social interactions network

In [ ]:
zero_weight_ties = social_net.loc[social_net.weight == 0,].index
social_net.drop(zero_weight_ties, inplace=True)
social_net.reset_index(drop=True, inplace=True)
social_net.info()

In [ ]:
social_net.weight.describe()

In [ ]:
G_social = nx.from_pandas_edgelist(social_net,
                                   source='from',
                                   target='to',
                                   edge_attr='weight',
                                   create_using=nx.DiGraph)
print(G_social)

In [ ]:
warnings.simplefilter('ignore', UserWarning)

plot_graph(G_social, "Social interactions network", edge_weight_multiplier=0.55)

Note that the teacher (#16) is quite centrally positioned, whereas the isolated student (#4) is also the one missing from the friendship network.

Examine the highly connected nodes:

In [ ]:
print(G_social.get_edge_data(9,12).get('weight')) # one way of accessing a specific edge in a graph
print(G_social.get_edge_data(12,9).get('weight'))
print(G_social[2][13]['weight']) # another way of accessing an edge
print(G_social[13][2]['weight'])

Compute density, local clustering coefficient and transitivity

In [ ]:
social_net_density = nx.density(G_social)
social_net_density

In [ ]:
social_trans_score = nx.transitivity(G_social)

print(f"Social interactions network's transitivity score: {social_trans_score:.4f}")

In [ ]:
social_local_clust_coef = nx.clustering(G_social)

plot_graph(G_social,
           "Social interactions network, node color denotes local clustering coefficient",
           node_color_modifiers=social_local_clust_coef,
           edge_weight_multiplier=0.55)

In [ ]:
print(pd.Series(social_local_clust_coef.values()).mean())

Higher average local clustering compared to the global clustering (transitivity) coefficient suggests that low connected nodes cluster together more than well-connected nodes.

#### Task interactions network

In [ ]:
zero_weight_ties = task_net.loc[task_net.weight == 0,].index
task_net.drop(zero_weight_ties, inplace=True)
task_net.reset_index(drop=True, inplace=True)
task_net.info()

In [ ]:
task_net.weight.describe()

In [ ]:
G_task = nx.from_pandas_edgelist(task_net,
                                 source='from', target='to', edge_attr='weight', create_using=nx.DiGraph)
print(G_task)

In [ ]:
warnings.simplefilter('ignore', UserWarning)

plot_graph(G_task, "Task interaction network", edge_weight_multiplier=0.75)

Compute density, local clustering coefficient and transitivity

In [ ]:
task_net_density = nx.density(G_task)
print(task_net_density)

In [ ]:
task_trans_score = nx.transitivity(G_task)

print(f"Task interactions network's transitivity score: {task_trans_score:.4f}")

In [ ]:
task_local_clust_coef = nx.clustering(G_task)

plot_graph(G_task,
           "Task interactions network, node color denotes local clustering coefficient",
           node_color_modifiers=task_local_clust_coef,
           edge_weight_multiplier=0.75)

In [ ]:
print(pd.Series(task_local_clust_coef.values()).mean())

The significant difference between the two measures indicates (and the plot confirms) that not much connected nodes cluster together much more than well-connected nodes

#### Network comparison in terms of density, transitivity and avg. local clustering

In [ ]:
from statistics import median, mean

connectedmess = pd.DataFrame({
    "network": ["friendship", "social_inter", "task_inter"],
    "n_edges": [G_friend.number_of_edges(), G_social.number_of_edges(), G_task.number_of_edges()],
    "density": [fr_density, social_net_density, task_net_density],
    "transitivity": [fr_trans_score, social_trans_score, task_trans_score],
    "avg_clust_coef": [mean(fr_clust_coef.values()), mean(social_local_clust_coef.values()), mean(task_local_clust_coef.values())]
})

In [ ]:
connectedmess

## Community Detection

#### Transformation to undirected network

We'll use the friend network as the basis for our exploration of community detection methods.
For simplicity, we'll set the network to undirected and ensure it is connected. This is because most of the current community detection algorithms work only with undirected, connected graphs.

To make a distinction between reciprocated and non-reciprocated friend / best-friend ties, when transforming the graph to undirected, we will sum the weights of reciprocated edges

In [ ]:
G_friend_nondir = nx.Graph()

for u, v, attr in G_friend.edges(data=True):
    w = attr.get('weight', 0)
    if G_friend_nondir.has_edge(u, v):
        G_friend_nondir[u][v]['weight'] += w
    else:
        G_friend_nondir.add_edge(u, v, weight=w)

In [ ]:
print(G_friend_nondir)

Note that the number of edges has decreased (62->42) as reciprocated directed ties were consolidated into single undirected ties.

In [ ]:
edge_weights = []
for _, _, attr in G_friend_nondir.edges(data=True):
    edge_weights.append(attr['weight'])

pd.Series(edge_weights).value_counts()

In [ ]:
warnings.simplefilter('ignore', UserWarning)

plot_graph(G_friend_nondir, "Friend network (undirected)", edge_weight_multiplier=1.2)

In [ ]:
fr_undir_avg_clust_coeff = nx.average_clustering(G_friend_nondir)
fr_undir_trans_score = nx.transitivity(G_friend_nondir)

print(f"Undirected friend network:\n(i) average clustering coefficient: {fr_undir_avg_clust_coeff:.4f},\n(ii) transitivity score: {fr_undir_trans_score:.4f}")

Higher clustering than in the directed version of the network (expected), but remained fairly even among well-connected and low connected nodes.

#### Community detection methods and modularity

There are many different ways to detect communities in a network. A common characteristic of a large majority, if not all, of them is their reliance on a measure called **Modularity** to estimate the quality of the detected community structure.

Modularity measures the strength of division of a network into modules (clusters, communities). Networks with high modularity have dense connections among nodes within modules (communities) and sparse connections among nodes in different modules.
Modularity takes values between -0.5 and 1. If the whole network is a single community, modularity is zero; it is negative if each node is a community on its own.

Note: for more details about the modularity measure, listen / watch the explanation of modularity given in the (video) lecture: [Network Analysis. Lecture 8. Network communities](https://www.youtube.com/watch?v=lU1QEUH0nNc) around 1h 5min

In this lab, we'll examine the following algorithms for community detection:
* Louvain (Multi-level optimization of modularity)
* Edge-betweenness (Girvan & Newman)
* Walktrap

Eventually, we will compare their performance using the Modularity metric. Thus, we'll keep a list of modularity scores produced by the algorithms:

In [ ]:
modularity_scores = {}

#### Louvain community detection method

The Louvaine community detection method is a hierarchical, bottom-up process, based on modularity.
In brief, it works as follows:
* Initially, each vertex is assigned to a community on its own.
* In every step, vertices are re-assigned to communities as follows:
  * Each community is considered as a vertex on its own;
  * Each vertex is moved to the community with which it achieves the highest contribution to modularity;
  * When no vertices can be reassigned, so that the reassignment further maximizes modularity, each community is considered a vertex, and the process starts again with the merged communities (as new vertices).
* The process stops when there is only a single vertex left or when the modularity cannot be increased further in the given step.

The overall process somewhat resembles hierarchical agglomerative clustering that we did before as a part of machine learning clustering methods.

Main advantages: it's fast, works with weighted graphs and produces hierarchical clusters

In [ ]:
fr_louvaine_communities = nx.community.louvain_communities(G_friend_nondir, weight='weight')

print(f"Detected communities:{fr_louvaine_communities}")

Create a dictionary of nodes as keys and the communities they were asssigned to as values; this will be used as an input for visualisation

In [ ]:
fr_partition = {}
for i, community in enumerate(fr_louvaine_communities):
    for node in community:
        fr_partition[node] = i

In [ ]:
plot_graph(G_friend_nondir,
           "Communities detected in undirected Friendship network, with Louvaine method",
           node_color_modifiers=fr_partition,
           edge_weight_multiplier=1.5)

Get the modularity score for the detected communities:

In [ ]:
modularity_scores['Louvain'] = nx.community.modularity(G_friend_nondir, fr_louvaine_communities)
print(modularity_scores['Louvain'])

This is a fairly high value, suggesting well split into communities, as the above graph confirms

#### Edge-betweenness community detection method

The edge-betweenness (EB) score of an edge measures the proportion of shortest paths between any pair of vertices in the graph that go through that edge.

The EB community detection method (also known as ***Girvan & Newman*** method) is a hierarchical graph decomposition process where edges are removed in the decreasing order of their EB scores. The method is motivated by the assumption that edges connecting different communities are more likely to be part of multiple shortest paths, that is, have high EB score, simply because in many cases they are the best or the only option to go from one community to another.

In networkX, this method is implemented in the `nx.community.girvan_newman()` function.
This function returns an iterator over tuples where each tuple represents one level / iteration in community splitting:
* The first tuple contains the initial split, after the first edge removal
* Each subsequent tuple represents further division until no edges remain

To select the optimal partition, we should iterate through the potential community splits and identify the partition that yields the maximum modularity score:

In [ ]:
fr_eb_communities = list(nx.community.girvan_newman(G_friend_nondir))

for partition in fr_eb_communities:
    print(partition)

In [ ]:
for partition in fr_eb_communities:
    m = nx.community.modularity(G_friend_nondir, partition)
    print(f"{partition} ----> {m:.3f}")

In [ ]:
fr_eb_optimal = fr_eb_communities[1]
fr_eb_optimal

In [ ]:
modularity_scores['EB'] = nx.community.modularity(G_friend_nondir, fr_eb_optimal)

In [ ]:
fr_eb_partition = {}
for i, community in enumerate(fr_eb_optimal):
    for node in community:
        fr_eb_partition[node] = i

In [ ]:
plot_graph(G_friend_nondir,
           "Communities in undirected Friendship network, using Edge-betweenness method",
           node_color_modifiers=fr_eb_partition,
           edge_weight_multiplier=1.5)

#### Walktrap community detection method

This algorithm detects communities in a bottom-up manner, through a series of short random walks through the graph. The idea is that the vertices encountered on any given random walk are more likely to be within a community than not, since there are typically only a few edges that lead outside a given community.

To group nodes, the algorithm needs to measure how "close" / "distant" two nodes are. To this end, it uses the structural similarity / distance of short random walks of those nodes. We say that two nodes are similar if the probabilities of reaching any other node $k$ in $t$ steps is very similar.

In brief, the algorithm works as follows:
1) Initialization: Every node starts in its own individual community.

2) Calculate Distances: It computes the distances between all adjacent communities (initially, this is just the distances between connected nodes). Distances are computed based on the notion of structural distnace of the nodes' random walks.

3) Iterative Merging: In each step, it chooses the two communities that, when merged, minimize the increase in the "mean squared distance" of the partition. This is similar to Ward's method in traditional clustering.

4) Dendrogram: This process continues until all nodes are in a single giant community, creating a dendrogram.

5) Selection: To find the "best" partition, the algorithm usually looks for the step in the dendrogram that maximizes a quality metric like Modularity.

The overall process quite resembles the Louvain algorithm; the main difference is in the criterion for grouping nodes into communities: structural similarity / distance in Walktrap vs. modularity in Louvain.

The Walktrap community detection algorithm is not implemented in the networkx library. To use it, we will use the `igraph` library. This will require that we first convert our networkx graph to the format required by igraph.

In [ ]:
# Convert the networkx graph to an igraph graph
edges_as_tuple_list = [(u, v, attr['weight']) for u, v, attr in G_friend_nondir.edges(data=True)]

G_friend_nondir_ig = ig.Graph.TupleList(edges_as_tuple_list, directed=False, weights=True)
print(G_friend_nondir_ig)

In [ ]:
# Run the Walktrap algorithm
fr_walktrap_communities = G_friend_nondir_ig.community_walktrap(weights='weight')

# Get the optimal number of communities based on modularity
fr_walktrap_optimal = fr_walktrap_communities.as_clustering()
print(f"Optimal communities: {fr_walktrap_optimal}")

In [ ]:
modularity_scores['Walktrap'] = fr_walktrap_optimal.modularity

In [ ]:
fr_walktrap_membership = fr_walktrap_optimal.membership
print(f"Community membership for nodes: {fr_walktrap_membership}")

In [ ]:
fr_walktrap_partition = {node:membership
                         for node, membership in zip(G_friend_nondir_ig.vs['name'], fr_walktrap_membership)}

In [ ]:
plot_graph(G_friend_nondir,
           "Communities detected in undirected Friendship network, with Walktrap method",
           node_color_modifiers=fr_walktrap_partition,
           edge_weight_multiplier=1.5)

In [ ]:
print(modularity_scores)

All three algorithms gave the same result, which is a strong confirmation of the quality of this network partition.

Finally, we will try to identify network connectors as nodes with high betweenness centrality that are are at the edges of the identified communities

In [ ]:
fr_betweenness = nx.betweenness_centrality(G_friend_nondir, weight='weight')

In [ ]:
def plot_graph_advanced(G, graph_name, node_color_modifiers, node_size_modifiers, edge_weight_multiplier=1):
    plt.figure(figsize=(8,8))

    pos = nx.spring_layout(G, seed=9)

    node_color = [node_color_modifiers[node] for node in G.nodes()]
    node_size = [200 + 1500*node_size_modifiers[node] for node in G.nodes()]
    edge_width = [attr['weight']*1.5 for (u, v, attr) in G.edges(data=True)]

    nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=node_color, cmap='viridis')
    nx.draw_networkx_edges(G, pos, width=edge_width, edge_color='steelblue')
    nx.draw_networkx_labels(G, pos, font_color='ivory')

    plt.title(graph_name)

    plt.axis('off')
    plt.show()

In [ ]:
plot_graph_advanced(G_friend_nondir,
                    "Communities in undirected Friendship network",
                    node_color_modifiers=fr_eb_partition,
                    node_size_modifiers=fr_betweenness,
                    edge_weight_multiplier=1.5)

Actors 1, 15, and 14 have important positions as network connectors.